# Data Mining 2 Project

To do:
1. Figure out how to get one table into a data frame: DONE
2. Save the data frame to CSV file: DONE
3. Examine the list of lists: DONE
4. Identify columns to remove
5. Build new query excluding unwanted columns
6. Start downloading data to CSV

==
Examine assumptions required for statistical modeling plan:
- Pros of planned methods
- Cons of planned methods


## Get Access To the Data and set up libaries

In [1]:
import bq_helper
import pandas as pd
import os
#from bq_helper import BigQueryHelper

# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
# https://github.com/SohierDane/BigQuery_Helper/blob/master/bq_helper.py

# Establish Helper Object for data scanning
google_analytics = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="google_analytics_sample")

# Another example of how to get the data
# bq_assistant = BigQueryHelper("bigquery-public-data", "google_analytics_sample")

# Create list of tables to later assist with queries
tablelist = google_analytics.list_tables()
print(len(tablelist))
print("First table:", tablelist[0],"  Last table:", tablelist[-1])

366
First table: ga_sessions_20160801   Last table: ga_sessions_20170801


The above table list confirms that the data start at August 1, 2016 (20160801) and end at August 1, 2017 (20170801)

Now examine the overall structure of a table

In [2]:
print("Table ID:", tablelist[0])
print(google_analytics.head(tablelist[0]).columns)
google_analytics.head(tablelist[0], num_rows=3)

Table ID: ga_sessions_20160801
Index(['visitorId', 'visitNumber', 'visitId', 'visitStartTime', 'date',
       'totals', 'trafficSource', 'device', 'geoNetwork', 'customDimensions',
       'hits', 'fullVisitorId', 'userId', 'channelGrouping',
       'socialEngagementType'],
      dtype='object')


,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged


In [3]:
print("Table ID:", tablelist[0])
google_analytics.table_schema(tablelist[0])

Table ID: ga_sessions_20160801


[SchemaField('visitorId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitNumber', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitId', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('visitStartTime', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('date', 'STRING', 'NULLABLE', None, ()),
 SchemaField('totals', 'RECORD', 'NULLABLE', None, (SchemaField('visits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('hits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('pageviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnSite', 'INTEGER', 'NULLABLE', None, ()), SchemaField('bounces', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactions', 'INTEGER', 'NULLABLE', None, ()), SchemaField('transactionRevenue', 'INTEGER', 'NULLABLE', None, ()), SchemaField('newVisits', 'INTEGER', 'NULLABLE', None, ()), SchemaField('screenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('uniqueScreenviews', 'INTEGER', 'NULLABLE', None, ()), SchemaField('timeOnScreen', 'INTEGER', 'NULLAB

## Queries

### Total Size of All Data
First, I'll find the size of the entire table in Gigabytes by using a funciton native to the bq_helper package

In [4]:
# Determine Total Datasize
query_everything = """
#standardSQL
SELECT *
FROM 
    # enclose table names with WILDCARDS in backticks `` , not quotes ''
    `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
    _TABLE_SUFFIX < '20170802'
"""
google_analytics.estimate_query_size(query_everything)

5.370753524824977

### Select 1 table

1. Start by esimtating the query size
2. If query size is acceptable, save as data frame

In [5]:
query_oneTable = """
#standardSQL
SELECT *
FROM 
    # enclose table names with WILDCARDS in backticks `` , not quotes ''
    `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`
"""
google_analytics.estimate_query_size(query_oneTable)

0.01855206210166216

In [6]:
oneTable = google_analytics.query_to_pandas_safe(query_oneTable, max_gb_scanned=.1)
oneTable.head(3)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,None,1,1470117657,1470117657,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Internet Explorer', 'browserVersi...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 23, 'minu...",7194065619159478122,None,Social,Not Socially Engaged
1,None,151,1470083489,1470083489,20160801,"{'visits': 1, 'hits': 3, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Americas', 'subContinent': 'Nor...","[{'index': 4, 'value': 'North America'}]","[{'hitNumber': 1, 'time': 0, 'hour': 13, 'minu...",8159312408158297118,None,Social,Not Socially Engaged
2,None,1,1470052694,1470052694,20160801,"{'visits': 1, 'hits': 4, 'pageviews': 3, 'time...","{'referralPath': '/yt/about/', 'campaign': '(n...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Asia', 'subContinent': 'Southea...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 4, 'minut...",9236304747882138291,None,Social,Not Socially Engaged


In [7]:
print(oneTable.shape, oneTable.columns)

(1711, 15) Index(['visitorId', 'visitNumber', 'visitId', 'visitStartTime', 'date',
       'totals', 'trafficSource', 'device', 'geoNetwork', 'customDimensions',
       'hits', 'fullVisitorId', 'userId', 'channelGrouping',
       'socialEngagementType'],
      dtype='object')


### Examine Column Names

In [8]:
for col in oneTable.columns:
    print(col, ": ", type(oneTable[col][0]))
    print(oneTable[col][0])
    #print(oneTable[col])
    

visitorId :  <class 'NoneType'>
None
visitNumber :  <class 'numpy.int64'>
1
visitId :  <class 'numpy.int64'>
1470117657
visitStartTime :  <class 'numpy.int64'>
1470117657
date :  <class 'str'>
20160801
totals :  <class 'dict'>
{'visits': 1, 'hits': 3, 'pageviews': 3, 'timeOnSite': 54, 'bounces': None, 'transactions': None, 'transactionRevenue': None, 'newVisits': 1, 'screenviews': None, 'uniqueScreenviews': None, 'timeOnScreen': None, 'totalTransactionRevenue': None, 'sessionQualityDim': None}
trafficSource :  <class 'dict'>
{'referralPath': '/yt/about/', 'campaign': '(not set)', 'source': 'youtube.com', 'medium': 'referral', 'keyword': None, 'adContent': None, 'adwordsClickInfo': {'campaignId': None, 'adGroupId': None, 'creativeId': None, 'criteriaId': None, 'page': None, 'slot': None, 'criteriaParameters': 'not available in demo dataset', 'gclId': None, 'customerId': None, 'adNetworkType': None, 'targetingCriteria': None, 'isVideoAd': None}, 'isTrueDirect': None, 'campaignCode': None

## Test writing to CSV

In [9]:
# myfilepath = "C:/Users/Jusitn/Documents/Python Scripts/DataMining2"
oneTable.to_csv("20160801.csv", encoding='utf-8', index=False)

## Test Reading from CSV
**Important Note:** after perfomring the 'Test writing to CSV', I had to click on the 'Data' tab in the Kernel, look at the left hand column, and add my own kernel as a data source so that i could see the CSV files in my directory.
THEN, be sure to select the table and click "download"

In [10]:
import pandas as pd
googledata = pd.read_csv('../input/20160801.csv')
#DOES NOT WORK:  pd.read_csv('../input/data-mining-2-project-in-python/20160801.csv')

In [ ]:
googledata.head(1)

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,NaN,1,1470046245,1470046245,20160801,"{'visits': 1, 'hits': 24, 'pageviews': 17, 'ti...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Firefox', 'browserVersion': 'not ...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 3, 'minut...",895954260133011192,NaN,Organic Search,Not Socially Engaged


In [ ]:
for col in googledata.columns:
    print(col, ": ", type(googledata[col][0]))
    print(googledata[col][0])
    #print(oneTable[col])

The above data 

In [ ]:
googledata['trafficSource'][0]

# Example Queries

What is the total number of transactions generated per device browser in July 2017?

query1 = """SELECT
device.browser,
SUM ( totals.transactions ) AS total_transactions
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20160801' AND '20170801'
GROUP BY
device.browser
ORDER BY
total_transactions DESC;
        """
google_analytics.estimate_query_size(query1)
#response1 = google_analytics.query_to_pandas_safe(query1)
#response1.head(10)


The real bounce rate is defined as the percentage of visits with a single pageview. What was the real bounce rate per traffic source?

query2 = """SELECT
source,
total_visits,
total_no_of_bounces,
( ( total_no_of_bounces / total_visits ) * 100 ) AS bounce_rate
FROM (
SELECT
trafficSource.source AS source,
COUNT ( trafficSource.source ) AS total_visits,
SUM ( totals.bounces ) AS total_no_of_bounces
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
GROUP BY
source )
ORDER BY
total_visits DESC;
        """
response2 = google_analytics.query_to_pandas_safe(query2)
response2.head(10)

What is the average amount of money spent per session in July 2017?

query6 = """SELECT
( SUM(total_transactionrevenue_per_user) / SUM(total_visits_per_user) ) AS
avg_revenue_by_user_per_visit
FROM (
SELECT
fullVisitorId,
SUM( totals.visits ) AS total_visits_per_user,
SUM( totals.transactionRevenue ) AS total_transactionrevenue_per_user
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
totals.visits > 0
AND totals.transactions >= 1
AND totals.transactionRevenue IS NOT NULL
GROUP BY
fullVisitorId );
        """
response6 = google_analytics.query_to_pandas_safe(query6, max_gb_scanned=10)
response6.head(10)

What is the sequence of pages viewed?

query7 = """SELECT
fullVisitorId,
visitId,
visitNumber,
hits.hitNumber AS hitNumber,
hits.page.pagePath AS pagePath
FROM
`bigquery-public-data.google_analytics_sample.ga_sessions_*`,
UNNEST(hits) as hits
WHERE
_TABLE_SUFFIX BETWEEN '20170701' AND '20170731'
AND
hits.type="PAGE"
ORDER BY
fullVisitorId,
visitId,
visitNumber,
hitNumber;
        """
response7 = google_analytics.query_to_pandas_safe(query7, max_gb_scanned=10)
response7.head(10)